In [ ]:
!az login -o table

In [ ]:
!az provider register -n Microsoft.BatchAI
!az provider register -n Microsoft.Batch

In [1]:
!az account list -o table

Name                                 CloudName    SubscriptionId                        State    IsDefault
-----------------------------------  -----------  ------------------------------------  -------  -----------
Marketing Automation                 AzureCloud   03909a66-bef8-4d52-8e9a-a346604e0902  Enabled  True
Boston Engineering                   AzureCloud   bc4170f0-cc6e-49d2-ba65-bc00a7a4df6b  Enabled
ADLTrainingMS                        AzureCloud   ace74b35-b0de-428b-a1d9-55459d7a6e30  Enabled
Ads Eng Big Data Subscription        AzureCloud   d4231148-f093-4473-85de-8c4d19247495  Enabled
Data Wrangling Preview               AzureCloud   215613ac-9dfb-488c-be46-c387e999b127  Enabled
Data Wrangling development           AzureCloud   35f16a99-532a-4a47-9e93-00305f6c40f2  Enabled
AzureML Client PROD                  AzureCloud   4a0f8d7f-c197-476b-95e1-fafcd77f12e4  Enabled
R portal - Production                AzureCloud   2569dcd9-a995-4b78-ae22-58a1015fd0f7  Enabled
MSEG_11036

In [ ]:
!az account set -s "<YOURTEAM>"

In [2]:
!az --version

azure-cli (2.0.32)

acr (2.0.24)
acs (2.0.32)
advisor (0.5.1)
ams (0.1.0)
appservice (0.1.32)
backup (1.1.1)
batch (3.2.2)
batchai (0.2.2)
billing (0.1.8)
cdn (0.0.14)
cloud (2.0.13)
cognitiveservices (0.1.13)
command-modules-nspkg (2.0.1)
configure (2.0.15)
consumption (0.3.1)
container (0.1.23)
core (2.0.32)
cosmosdb (0.1.21)
dla (0.0.19)
dls (0.0.21)
dms (0.0.1)
eventgrid (0.1.12)
eventhubs (0.1.3)
extension (0.0.13)
feedback (2.1.1)
find (0.2.9)
interactive (0.3.20)
iot (0.1.20)
keyvault (2.0.21)
lab (0.0.22)
monitor (0.1.6)
network (2.1.0)
nspkg (3.0.2)
profile (2.0.24)
rdbms (0.2.2)
redis (0.2.13)
reservations (0.1.2)
resource (2.0.28)
role (2.0.23)
servicebus (0.1.2)
servicefabric (0.0.12)
sql (2.0.25)
storage (2.0.32)
vm (2.0.31)

Python location '/opt/az/bin/python3'
Extensions directory '/home/miguel/.azure/cliextensions'

Python (Linux) 3.6.1 (default, Apr 30 2018, 23:14:03) 
[GCC 5.4.0 20160609]

Legal docs and information: aka.ms/AzureCliLegal




In [3]:
import os
import uuid
import random
import json

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

LOCATION = 'eastus' # We are setting everything up in East US
                    # Be aware that you need to set things up in a region that has GPU VMs (N-Series)

short_uuid = str(uuid.uuid4())[:8]
GROUP_NAME = "batch{uuid}rg".format(uuid=short_uuid)
STORAGE_ACCOUNT_NAME = "batch{uuid}st".format(uuid=short_uuid)
CONTAINER_NAME = "batch{uuid}container".format(uuid=short_uuid)
FILESHARE_NAME = "batch{uuid}share".format(uuid=short_uuid)
NFS_NAME="batch{uuid}nfs".format(uuid=short_uuid)

In [4]:
!az group create -n $GROUP_NAME -o table -l $LOCATION

Location    Name
----------  ---------------
eastus      batch47389c82rg


In [5]:
json_data = !az storage account create -l $LOCATION -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME --sku Standard_LRS
print('Storage account {} provisioning state: {}'.format(STORAGE_ACCOUNT_NAME, json.loads(''.join(json_data))['provisioningState']))

Storage account batch47389c82st provisioning state: Succeeded


In [6]:
json_data = !az storage account keys list -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME
storage_account_key = json.loads(''.join([i for i in json_data if 'WARNING' not in i]))[0]['value']

In [7]:
!az storage container create --account-name $STORAGE_ACCOUNT_NAME \
--account-key $storage_account_key --name $CONTAINER_NAME

{
  "created": true
}


In [8]:
!az storage share create --account-name $STORAGE_ACCOUNT_NAME \
--account-key $storage_account_key --name $FILESHARE_NAME

{
  "created": true
}


In [9]:
!az storage directory create --share-name $FILESHARE_NAME  --name scripts \
--account-name $STORAGE_ACCOUNT_NAME --account-key $storage_account_key

{
  "created": true
}


In [10]:
!az configure --defaults location=$LOCATION

In [11]:
!az configure --defaults group=$GROUP_NAME

In [12]:
%env AZURE_STORAGE_ACCOUNT $STORAGE_ACCOUNT_NAME

env: AZURE_STORAGE_ACCOUNT=batch47389c82st


In [13]:
%env AZURE_STORAGE_KEY=$storage_account_key

env: AZURE_STORAGE_KEY=pagaSJpBSTJNY/dAaTexrPxHp/3lZcyzP6v01f5PvtK7oECom2j6HGC+Cb0EPh2cx7Nuy1V1Hnz0HUqWj+4cOA==


In [14]:
destination_url = "https://{}.blob.core.windows.net/{}/{}".format(STORAGE_ACCOUNT_NAME, 
                                                                  CONTAINER_NAME,
                                                                  'validation')

In [15]:
!azcopy --source https://datasharesa.blob.core.windows.net/imagenet/validation \
        --source-key 'owUPSqTbwAigV54BHTr8oYABEha8xi/VsA4HD06GboDgOb3pf6OFgtw/tlKYv/AlkgSIBkxqoA28hnkIeo4NFg==' \
        --destination  $destination_url \
        --dest-key $storage_account_key --quiet --recursive 

=Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 98.477 KB; Average Speed:7.08 KB/s.                       Finished: 3 file(s), 410.15 KB; Average Speed:25.71 KB/s.                      Finished: 8 file(s), 928.097 KB; Average Speed:51.56 KB/s.                     Finished: 13 file(s), 1.368 MB; Average Speed:69.85 KB/s.                      Finished: 16 file(s), 1.72 MB; Average Speed:79.69 KB/s.                       Finished: 21 file(s), 2.221 MB; Average Speed:94.21 KB/s.                      Finished: 24 file(s), 2.58 MB; Average Speed:100.84 KB/s.                      Finished: 28 file(s), 3.109 MB; Average Speed:112.72 KB/s.                     Finished: 30 file(s), 3.46 MB; Average Speed:116.9

Finished: 10661 file(s), 1.301 GB; Average Speed:7.16 MB/s.                    Finished: 10802 file(s), 1.32 GB; Average Speed:7.19 MB/s.                     Finished: 10939 file(s), 1.336 GB; Average Speed:7.2 MB/s.                     Finished: 11100 file(s), 1.357 GB; Average Speed:7.23 MB/s.                    Finished: 11256 file(s), 1.376 GB; Average Speed:7.26 MB/s.                    Finished: 11420 file(s), 1.396 GB; Average Speed:7.29 MB/s.                    Finished: 11587 file(s), 1.415 GB; Average Speed:7.31 MB/s.                    Finished: 11749 file(s), 1.436 GB; Average Speed:7.34 MB/s.                    Finished: 11898 file(s), 1.454 GB; Average Speed:7.36 MB/s.                    Finished: 12032 file(s), 1.469 GB; Average Speed:7.36 MB/s.                    Finished: 12175 file(s), 1.487 GB; Average Speed:7.38 MB/s.                    Finished: 12315 file(s), 1.507 GB; Average Speed:7.4 MB/s.                     Finished: 12462 file(s), 1.526 GB; Average Speed:7.4

Finished: 24258 file(s), 3.02 GB; Average Speed:8.4 MB/s.                      Finished: 24387 file(s), 3.038 GB; Average Speed:8.4 MB/s.                     Finished: 24542 file(s), 3.057 GB; Average Speed:8.41 MB/s.                    Finished: 24681 file(s), 3.074 GB; Average Speed:8.41 MB/s.                    Finished: 24841 file(s), 3.092 GB; Average Speed:8.41 MB/s.                    Finished: 24990 file(s), 3.11 GB; Average Speed:8.41 MB/s.                     Finished: 25155 file(s), 3.129 GB; Average Speed:8.42 MB/s.                    Finished: 25296 file(s), 3.146 GB; Average Speed:8.42 MB/s.                    Finished: 25442 file(s), 3.165 GB; Average Speed:8.43 MB/s.                    Finished: 25585 file(s), 3.183 GB; Average Speed:8.43 MB/s.                    Finished: 25733 file(s), 3.201 GB; Average Speed:8.43 MB/s.                    Finished: 25881 file(s), 3.22 GB; Average Speed:8.44 MB/s.                     Finished: 26042 file(s), 3.239 GB; Average Speed:8.4

Finished: 37923 file(s), 4.728 GB; Average Speed:8.83 MB/s.                    Finished: 38080 file(s), 4.748 GB; Average Speed:8.83 MB/s.                    Finished: 38252 file(s), 4.768 GB; Average Speed:8.84 MB/s.                    Finished: 38412 file(s), 4.789 GB; Average Speed:8.84 MB/s.                    Finished: 38539 file(s), 4.807 GB; Average Speed:8.84 MB/s.                    Finished: 38664 file(s), 4.823 GB; Average Speed:8.84 MB/s.                    Finished: 38808 file(s), 4.841 GB; Average Speed:8.84 MB/s.                    Finished: 38946 file(s), 4.857 GB; Average Speed:8.84 MB/s.                    Finished: 39092 file(s), 4.875 GB; Average Speed:8.84 MB/s.                    Finished: 39232 file(s), 4.892 GB; Average Speed:8.84 MB/s.                    Finished: 39375 file(s), 4.91 GB; Average Speed:8.84 MB/s.                     Finished: 39517 file(s), 4.928 GB; Average Speed:8.84 MB/s.                    Finished: 39621 file(s), 4.941 GB; Average Speed:8.8

                                                                               Finished 49999 of total 50000 file(s).
                                                                               [2018/06/16 07:46:31] Transfer summary:
-----------------
Total files transferred: 50000
Transfer successfully:   49999
Transfer skipped:        0
Transfer failed:         1
Elapsed time:            00.00:19:08


In [16]:
destination_url = "https://{}.blob.core.windows.net/{}/{}".format(STORAGE_ACCOUNT_NAME, 
                                                                  CONTAINER_NAME,
                                                                  'train')

In [ ]:
!azcopy --source https://datasharesa.blob.core.windows.net/imagenet/train \
        --source-key 'owUPSqTbwAigV54BHTr8oYABEha8xi/VsA4HD06GboDgOb3pf6OFgtw/tlKYv/AlkgSIBkxqoA28hnkIeo4NFg==' \
        --destination  $destination_url \
        --dest-key $storage_account_key --quiet --recursive --exclude-older

=                                                                               [2018/06/16 08:18:34][WARNING] The command line "--source "https://datasharesa.blob.core.windows.net/imagenet/validation" --destination "https://batch47389c82st.blob.core.windows.net/batch47389c82container/validation" --recursive --source-key ****** --dest-key ****** --quiet" in the journal file "/home/miguel/Microsoft/Azure/AzCopy/AzCopy.jnl" is different from your input.
                                                                               [2018/06/16 08:18:36][WARNING] Incomplete operation with different command line detected at the journal directory "/home/miguel/Microsoft/Azure/AzCopy".


Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 0 B; Average Speed:0 B/s.                                 Finished: 0 file(s), 173.938 KB; Average Speed:7.57 KB/s.                      Finished: 3 file(s), 520.127 KB; Average Speed:20.81 KB/s.                     Finished: 10 file(s), 915.338 KB; Average Speed:33.85 KB/s.                    Finished: 15 file(s), 1.315 MB; Average Speed:46.3 KB/s.                       Finished: 19 file(s), 1.565 MB; Average Speed:51.49 KB/s.                      Finished: 23 file(s), 2.042 MB; Average Speed:63.01 KB/s.                      Finished: 25 file(s), 2.777 MB; Average Speed:80.71 KB/s.                      Finished: 26 file(s), 2.777 MB; Average Speed:76.27 KB/s.                      Finished: 30 file(s), 3.16 MB; Average Speed:82.28 K

Finished: 21038 file(s), 2.364 GB; Average Speed:4.21 MB/s.                    Finished: 21118 file(s), 2.366 GB; Average Speed:4.2 MB/s.                     Finished: 21174 file(s), 2.371 GB; Average Speed:4.2 MB/s.                     Finished: 21317 file(s), 2.388 GB; Average Speed:4.21 MB/s.                    Finished: 21445 file(s), 2.401 GB; Average Speed:4.22 MB/s.                    Finished: 21468 file(s), 2.404 GB; Average Speed:4.21 MB/s.                    Finished: 21628 file(s), 2.419 GB; Average Speed:4.22 MB/s.                    Finished: 21708 file(s), 2.427 GB; Average Speed:4.22 MB/s.                    Finished: 21942 file(s), 2.451 GB; Average Speed:4.25 MB/s.                    Finished: 21953 file(s), 2.451 GB; Average Speed:4.23 MB/s.                    Finished: 22060 file(s), 2.468 GB; Average Speed:4.24 MB/s.                    Finished: 22125 file(s), 2.47 GB; Average Speed:4.23 MB/s.                     Finished: 22149 file(s), 2.472 GB; Average Speed:4.2

Finished: 29074 file(s), 3.23 GB; Average Speed:4.35 MB/s.                     Finished: 29174 file(s), 3.241 GB; Average Speed:4.35 MB/s.                    Finished: 29272 file(s), 3.256 GB; Average Speed:4.36 MB/s.                    Finished: 29431 file(s), 3.267 GB; Average Speed:4.37 MB/s.                    Finished: 29467 file(s), 3.271 GB; Average Speed:4.36 MB/s.                    Finished: 29634 file(s), 3.289 GB; Average Speed:4.37 MB/s.                    Finished: 29639 file(s), 3.289 GB; Average Speed:4.36 MB/s.                    Finished: 29704 file(s), 3.295 GB; Average Speed:4.36 MB/s.                    Finished: 29771 file(s), 3.311 GB; Average Speed:4.37 MB/s.                    Finished: 29864 file(s), 3.314 GB; Average Speed:4.36 MB/s.                    Finished: 29971 file(s), 3.327 GB; Average Speed:4.36 MB/s.                    Finished: 30026 file(s), 3.335 GB; Average Speed:4.36 MB/s.                    Finished: 30169 file(s), 3.349 GB; Average Speed:4.3

Finished: 37106 file(s), 4.185 GB; Average Speed:4.51 MB/s.                    Finished: 37217 file(s), 4.187 GB; Average Speed:4.5 MB/s.                     Finished: 37310 file(s), 4.198 GB; Average Speed:4.5 MB/s.                     Finished: 37364 file(s), 4.204 GB; Average Speed:4.5 MB/s.                     Finished: 37510 file(s), 4.221 GB; Average Speed:4.51 MB/s.                    Finished: 37542 file(s), 4.226 GB; Average Speed:4.5 MB/s.                     Finished: 37800 file(s), 4.248 GB; Average Speed:4.51 MB/s.                    Finished: 37847 file(s), 4.254 GB; Average Speed:4.51 MB/s.                    Finished: 38050 file(s), 4.273 GB; Average Speed:4.52 MB/s.                    Finished: 38122 file(s), 4.279 GB; Average Speed:4.52 MB/s.                    Finished: 38185 file(s), 4.292 GB; Average Speed:4.52 MB/s.                    Finished: 38320 file(s), 4.3 GB; Average Speed:4.52 MB/s.                      Finished: 38361 file(s), 4.303 GB; Average Speed:4.5

...

Finished: 357648 file(s), 38.792 GB; Average Speed:4.41 MB/s.                  Finished: 357691 file(s), 38.796 GB; Average Speed:4.41 MB/s.                  Finished: 357763 file(s), 38.806 GB; Average Speed:4.41 MB/s.                  Finished: 357924 file(s), 38.818 GB; Average Speed:4.41 MB/s.                  Finished: 357967 file(s), 38.823 GB; Average Speed:4.41 MB/s.                  Finished: 358013 file(s), 38.835 GB; Average Speed:4.41 MB/s.                  Finished: 358146 file(s), 38.849 GB; Average Speed:4.41 MB/s.                  Finished: 358176 file(s), 38.852 GB; Average Speed:4.41 MB/s.                  Finished: 358193 file(s), 38.855 GB; Average Speed:4.41 MB/s.                  Finished: 358231 file(s), 38.859 GB; Average Speed:4.41 MB/s.                  Finished: 358371 file(s), 38.885 GB; Average Speed:4.41 MB/s.                  Finished: 358374 file(s), 38.885 GB; Average Speed:4.41 MB/s.                  Finished: 358458 file(s), 38.893 GB; Average Speed:4

Finished: 364824 file(s), 39.571 GB; Average Speed:4.4 MB/s.                   Finished: 364893 file(s), 39.575 GB; Average Speed:4.4 MB/s.                   Finished: 364955 file(s), 39.58 GB; Average Speed:4.4 MB/s.                    Finished: 365102 file(s), 39.594 GB; Average Speed:4.4 MB/s.                   Finished: 365134 file(s), 39.594 GB; Average Speed:4.4 MB/s.                   Finished: 365222 file(s), 39.602 GB; Average Speed:4.4 MB/s.                   Finished: 365333 file(s), 39.614 GB; Average Speed:4.4 MB/s.                   Finished: 365392 file(s), 39.619 GB; Average Speed:4.4 MB/s.                   Finished: 365470 file(s), 39.629 GB; Average Speed:4.4 MB/s.                   Finished: 365518 file(s), 39.637 GB; Average Speed:4.4 MB/s.                   Finished: 365587 file(s), 39.643 GB; Average Speed:4.4 MB/s.                   Finished: 365716 file(s), 39.658 GB; Average Speed:4.4 MB/s.                   Finished: 365809 file(s), 39.669 GB; Average Speed:4

Finished: 372499 file(s), 40.456 GB; Average Speed:4.41 MB/s.                  Finished: 372529 file(s), 40.466 GB; Average Speed:4.41 MB/s.                  Finished: 372648 file(s), 40.473 GB; Average Speed:4.41 MB/s.                  Finished: 372701 file(s), 40.48 GB; Average Speed:4.41 MB/s.                   Finished: 372764 file(s), 40.486 GB; Average Speed:4.41 MB/s.                  Finished: 372975 file(s), 40.512 GB; Average Speed:4.41 MB/s.                  Finished: 373027 file(s), 40.519 GB; Average Speed:4.41 MB/s.                  Finished: 373157 file(s), 40.533 GB; Average Speed:4.41 MB/s.                  Finished: 373193 file(s), 40.539 GB; Average Speed:4.41 MB/s.                  Finished: 373398 file(s), 40.571 GB; Average Speed:4.41 MB/s.                  Finished: 373494 file(s), 40.58 GB; Average Speed:4.41 MB/s.                   Finished: 373578 file(s), 40.602 GB; Average Speed:4.42 MB/s.                  Finished: 373683 file(s), 40.605 GB; Average Speed:4

Finished: 379645 file(s), 41.342 GB; Average Speed:4.42 MB/s.                  Finished: 379710 file(s), 41.348 GB; Average Speed:4.42 MB/s.                  Finished: 379738 file(s), 41.351 GB; Average Speed:4.42 MB/s.                  Finished: 379889 file(s), 41.364 GB; Average Speed:4.42 MB/s.                  Finished: 379961 file(s), 41.372 GB; Average Speed:4.42 MB/s.                  Finished: 379980 file(s), 41.374 GB; Average Speed:4.42 MB/s.                  Finished: 380019 file(s), 41.38 GB; Average Speed:4.41 MB/s.                   Finished: 380195 file(s), 41.398 GB; Average Speed:4.42 MB/s.                  Finished: 380257 file(s), 41.405 GB; Average Speed:4.42 MB/s.                  Finished: 380348 file(s), 41.423 GB; Average Speed:4.42 MB/s.                  Finished: 380487 file(s), 41.431 GB; Average Speed:4.42 MB/s.                  Finished: 380506 file(s), 41.434 GB; Average Speed:4.42 MB/s.                  Finished: 380724 file(s), 41.456 GB; Average Speed:4

Finished: 387290 file(s), 42.21 GB; Average Speed:4.42 MB/s.                   Finished: 387392 file(s), 42.22 GB; Average Speed:4.42 MB/s.                   Finished: 387407 file(s), 42.222 GB; Average Speed:4.42 MB/s.                  Finished: 387552 file(s), 42.236 GB; Average Speed:4.42 MB/s.                  Finished: 387585 file(s), 42.24 GB; Average Speed:4.42 MB/s.                   Finished: 387672 file(s), 42.25 GB; Average Speed:4.42 MB/s.                   Finished: 387697 file(s), 42.253 GB; Average Speed:4.42 MB/s.                  Finished: 387783 file(s), 42.263 GB; Average Speed:4.42 MB/s.                  Finished: 387889 file(s), 42.273 GB; Average Speed:4.42 MB/s.                  Finished: 387936 file(s), 42.278 GB; Average Speed:4.42 MB/s.                  Finished: 387938 file(s), 42.281 GB; Average Speed:4.42 MB/s.                  Finished: 388094 file(s), 42.295 GB; Average Speed:4.42 MB/s.                  Finished: 388127 file(s), 42.299 GB; Average Speed:4

In [ ]:
destination_url = "https://{}.blob.core.windows.net/{}/{}".format(STORAGE_ACCOUNT_NAME, 
                                                                  CONTAINER_NAME,
                                                                  'train_map.txt')

In [ ]:
!azcopy --source https://datasharesa.blob.core.windows.net/imagenet/cntk/train_map.txt \
        --destination  $destination_url \
        --source-sas '?sp=r&st=2018-05-21T13:59:25Z&se=2025-05-21T21:59:25Z&spr=https&sv=2017-07-29&sig=Nl5%2FquqbqkaeJeZnM%2BNeX2T9mmMf%2BoLAJtV1OXugkdI%3D&sr=b' \
        --dest-key $storage_account_key --quiet


In [ ]:
destination_url = "https://{}.blob.core.windows.net/{}/{}".format(STORAGE_ACCOUNT_NAME, 
                                                                  CONTAINER_NAME,
                                                                  'val_map.txt')

In [ ]:
!azcopy --source https://datasharesa.blob.core.windows.net/imagenet/cntk/val_map.txt \
    --destination  $destination_url \
    --source-sas '?sp=r&st=2018-05-21T13:52:05Z&se=2025-05-21T21:52:05Z&spr=https,http&sv=2017-07-29&sig=YuPsiiPJ%2FUQTvs8U9i1uYuna8bQW5SiIxDGvl27wbyk%3D&sr=b'    \
    --dest-key $storage_account_key --quiet
    

In [ ]:
destination_url = "https://{}.blob.core.windows.net/{}/{}".format(STORAGE_ACCOUNT_NAME, 
                                                                  CONTAINER_NAME,
                                                                  'ImageNet1K_mean.xml')

In [ ]:
!azcopy --source https://datasharesa.blob.core.windows.net/imagenet/cntk/ImageNet1K_mean.xml \
        --destination  $destination_url \
        --source-sas '?sp=r&st=2018-05-21T14:00:22Z&se=2025-05-21T22:00:22Z&spr=https&sv=2017-07-29&sig=iZgfdPr1vgEHqLJPdOteBulaUQudh6etVH%2FzJ9wGGPU%3D&sr=b' \
        --dest-key $storage_account_key --quiet

In [ ]:
!az storage file upload --share-name $FILESHARE_NAME --source ../execution/ImagenetEstimatorCNTK.py --path scripts

In [ ]:
!az storage file upload --share-name $FILESHARE_NAME --source ../execution/resnet_models.py --path scripts

In [ ]:
!az batchai cluster create \
--name nc24r \
--image UbuntuLTS \
--vm-size Standard_NC24r \
--min 10 --max 10 \
--afs-name $FILESHARE_NAME \
--afs-mount-path extfs \
--container-name $CONTAINER_NAME \
--container-mount-path extcn \
--user-name mat \
--password dnstvxrz \
--storage-account-name $STORAGE_ACCOUNT_NAME \
--storage-account-key $storage_account_key 

In [ ]:
!az batchai cluster show -n nc24r

In [36]:
!az batchai cluster list -o table

Name    Resource Group    VM Size         State      Idle    Running    Preparing    Leaving    Unusable
------  ----------------  --------------  -------  ------  ---------  -----------  ---------  ----------
nc24r   batch47389c82rg   STANDARD_NC24r  steady       10          0            0          0           0


In [29]:
!az batchai cluster list-nodes -n nc24r -o table

ID                                  IP              Port
----------------------------------  ------------  ------
tvm-4155946844_1-20180616t161605z   40.121.43.64   50008
tvm-4155946844_10-20180616t161605z  40.121.43.64   50003
tvm-4155946844_2-20180616t161605z   40.121.43.64   50004
tvm-4155946844_3-20180616t161605z   40.121.43.64   50000
tvm-4155946844_4-20180616t161605z   40.121.43.64   50009
tvm-4155946844_5-20180616t161605z   40.121.43.64   50005
tvm-4155946844_6-20180616t161605z   40.121.43.64   50001
tvm-4155946844_7-20180616t161605z   40.121.43.64   50006
tvm-4155946844_8-20180616t161605z   40.121.43.64   50002
tvm-4155946844_9-20180616t161605z   40.121.43.64   50007


In [30]:
jobs_dict = {
  "$schema": "https://raw.githubusercontent.com/Azure/BatchAI/master/schemas/2018-03-01/job.json",
  "properties": {
    "nodeCount": 2,
    "cntkSettings": {
            "pythonScriptFilePath": "$AZ_BATCHAI_INPUT_SCRIPTS/ImagenetEstimatorCNTK.py",
            "processCount": 8
        },
    "environmentVariables": [{
        "name": "DISTRIBUTED",
        "value": "True"
    }],
    "stdOutErrPathPrefix": "$AZ_BATCHAI_MOUNT_ROOT/extfs",
    "inputDirectories": [{
        "id": "SCRIPTS",
        "path": "$AZ_BATCHAI_MOUNT_ROOT/extfs/scripts"
      },
      {
        "id": "TRAIN",
        "path": "$AZ_BATCHAI_MOUNT_ROOT/imagenet",
      },
      {
        "id": "TEST",
        "path": "$AZ_BATCHAI_MOUNT_ROOT/imagenet",
      },
    ],
    "outputDirectories": [{
        "id": "MODEL",
        "pathPrefix": "$AZ_BATCHAI_MOUNT_ROOT/extfs",
        "pathSuffix": "Models"
    }],
    "containerSettings": {
      "imageSourceRegistry": {
        "image": "microsoft/cntk:2.5.1-gpu-python3.5-cuda9.0-cudnn7.0"
      }
    }
  }
}

In [31]:
write_json_to_file(jobs_dict, 'job_blob.json')

In [37]:
!az batchai job create -n cntk_benchmark1b --cluster-name nc24r -c job_blob.json -g $GROUP_NAME

{- Finished ..
  "additionalProperties": {},
  "caffeSettings": null,
  "chainerSettings": null,
  "cluster": {
    "additionalProperties": {},
    "id": "/subscriptions/03909a66-bef8-4d52-8e9a-a346604e0902/resourceGroups/batch47389c82rg/providers/Microsoft.BatchAI/clusters/nc24r",
    "resourceGroup": "batch47389c82rg"
  },
  "cntkSettings": {
    "additionalProperties": {},
    "commandLineArgs": null,
    "configFilePath": null,
    "languageType": "Python",
    "processCount": 8,
    "pythonInterpreterPath": null,
    "pythonScriptFilePath": "$AZ_BATCHAI_INPUT_SCRIPTS/ImagenetEstimatorCNTK.py"
  },
  "constraints": {
    "additionalProperties": {},
    "maxWallClockTime": "7 days, 0:00:00"
  },
  "containerSettings": {
    "additionalProperties": {},
    "imageSourceRegistry": {
      "additionalProperties": {},
      "credentials": null,
      "image": "microsoft/cntk:2.5.1-gpu-python3.5-cuda9.0-cudnn7.0",
      "serverUrl": null
    }
  },
  "creationTime": "2018-06-17T06:15:16.6

In [33]:
!az batchai job list -o table

Name            Resource Group    Cluster    Cluster RG       Tool      Nodes  State
--------------  ----------------  ---------  ---------------  ------  -------  -------
cntk_benchmark  batch47389c82rg   nc24r      batch47389c82rg  cntk          2  running


In [ ]:
!az batchai job file stream -n cntk_benchmark -g $GROUP_NAME -f stderr.txt

In [ ]:
!az batchai job file stream -n cntk_benchmark -g $GROUP_NAME -f stdout.txt

In [ ]:
!az batchai job delete --name cntk_benchmark -y

In [ ]:
!az configure --defaults group=''
!az configure --defaults location=''

In [ ]:
!az batchai cluster delete --name nc24r -g $GROUP_NAME -y

In [ ]:
!az batchai file-server delete -n $NFS_NAME -y -g $GROUP_NAME

In [ ]:
!az group delete --name $GROUP_NAME -y